In [1]:
import pandas as pd

In [1]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query1 = """
with discount as(
select order_id, 
  case
  when sum(discount)!=0 then 1
  else 0
  end as has_discount
from yeti-dev-edw.aa_anon.aa_order_item
group by order_id )

select o.yeti_customer_uuid as customer_id,sum(has_discount) as has_discount
from discount d
left join `yeti-dev-edw.aa_anon.aa_order` o ON d.order_id=o.order_id
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2
"""

query2 = """
with join_color as(
with color as (
SELECT  product_id,
(case when color="Black" then 1 else 0 end) as black,
(case when color="Navy"  then 1 else 0 end) as navy,
(case when color="White" then 1 else 0 end) as white,
(case when color="Seafoam" then 1 else 0 end) as seafoam,
(case when color="Stainless" then 1 else 0 end) as stainless,
(case when color="Charcoal" then 1 else 0 end) as charcoal,
(case when color="Nordic Purple" then 1 else 0 end) as nordic_purple,
(case when color="Harvest Red" then 1 else 0 end) as harvest_red,
(case when color="Alpine Yellow" then 1 else 0 end) as alpine_yellow,
(case when color not in("White","Black","Navy","Seafoam","Stainless" ,"Charcoal","Nordic Purple","Harvest Red","Alpine Yellow") then 1 else 0 end) as other_color
from  `yeti-dev-edw.aa_anon.aa_product` p
)

select oi.order_id,oi.order_item_id,
sum(black)*sum(quantity) as black
,sum(navy)*sum(quantity) as navy
,sum(white)*sum(quantity) as white
,sum(seafoam)*sum(quantity) as seafoam
,sum(stainless)*sum(quantity) as stainless
,sum(charcoal)*sum(quantity) as charcoal
,sum(nordic_purple)*sum(quantity) as nordic_purple
,sum(harvest_red)*sum(quantity) as harvest_red
,sum(alpine_yellow)*sum(quantity) as alpine_yellow
,sum(other_color)*sum(quantity) as other_color


from color c
left join  `yeti-dev-edw.aa_anon.aa_order_item` oi on c.product_id=oi.product_id
group by oi.order_id,oi.order_item_id)

select o.yeti_customer_uuid as customer_id, sum (black) as black
,sum(navy) as navy
,sum(white) as white 
,sum(seafoam) as seafoam
,sum(stainless) as stainless
,sum(charcoal) as charcoal
,sum(nordic_purple) as nordic_purple
,sum(harvest_red) as harvest_red
,sum(alpine_yellow) as alpine_yellow
,sum(other_color) as other_color

from join_color jc 
left join  `yeti-dev-edw.aa_anon.aa_order` o on o.order_id=jc.order_id
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2
"""


query3 = """
SELECT o.yeti_customer_uuid as customer_id, SUM(oi.quantity) as lifetime_quantity,    sum(oi.quantity) / count(distinct o.order_id) as per_order_quantity,
FROM `yeti-dev-edw.aa_anon.aa_order_item` oi
LEFT JOIN `yeti-dev-edw.aa_anon.aa_order` o ON oi.order_id=o.order_id
#LEFT JOIN yeti-dev-edw.aa_anon.aa_customer customer ON o.yeti_customer_uuid=customer.yeti_customer_uuid
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2

"""

Main_query = """
with 

Product_percentage as (SELECT 
  customer.yeti_customer_uuid, 
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Other'or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Other_pct,
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Outdoor_Living_pct,
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Drinkware_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Cargo_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Soft_Cooler_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Bags_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Hard_Cooler_pct,

FROM yeti-dev-edw.aa_anon.aa_customer customer
LEFT JOIN yeti-dev-edw.aa_anon.aa_order orders using(yeti_customer_uuid)
LEFT JOIN yeti-dev-edw.aa_anon.aa_order_item orderItem USING (order_id)
LEFT JOIN yeti-dev-edw.aa_anon.aa_product product USING (product_id)
WHERE product_category IS NOT NULL
GROUP BY yeti_customer_uuid)

SELECT 
  customer.yeti_customer_uuid as customer_id, 
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(orders.creation_date)), DAY) / 30 AS recency,
  subquery.avg_time_btw_orders as avg_time_btw_orders,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(min(orders.creation_date)), DAY) / 30 AS months_elapsed,
  #sum(quantity) as lifetime_quantity,
  #sum (case when orderItem.discount>0 then quantity else 0 end) as test,
  #sum(quantity) / count(orders.order_id) as per_order_quantity,
  round(sum(orders.product_total)) as lifetime_spent,
  round(sum(orders.product_total))/ count(orders.order_id) as per_order_spent,

  avg(Outdoor_Living_pct) as outdoor_equipment_pct,
  avg(Cargo_pct) as Cargo_pct,
  avg(Soft_Cooler_pct) as Soft_Cooler_pct,
  avg(Bags_pct) as Bags_pct,
  avg(Drinkware_pct) as drinkware_pct,
  avg(Hard_Cooler_pct) as Hard_Cooler_pct,
  avg(Other_pct) as other_pct,
  #avg(has_discount) as order_with_discount,

  #avg(case when (bd) is null then 0 else bd  end ) as bag_discount_used,
  #avg(case when (od) is null then 0 else od  end ) as others_discount_used,
  #avg(case when (old) is null then 0 else old  end ) as outdoor_living_discount_used,
  #avg(case when (dd) is null then 0 else dd  end ) as drinkware_discount_used,
  #avg(case when (cd) is null then 0 else cd  end ) as cargo_discount_used,
  #avg(case when (scd) is null then 0 else scd  end ) as soft_cooler_discount_used,
  #avg(case when (hcd) is null then 0 else hcd  end ) as hard_cooler_discount_used,

  count(distinct orders.order_id) as purchase_times,
 
  COUNT(email_id) as email_received,
  COUNT(CASE WHEN last_open_date IS NOT NULL THEN 1 END) / NULLIF(COUNT(email_id),0) AS email_open_rate,
  CASE WHEN COUNT(email_id) = 0 THEN NULL ELSE MAX(CASE WHEN unsub_count > 0 THEN 1 ELSE 0 END) END AS unsubscribed,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(email.last_open_date)), DAY) / 30 AS months_since_last_email_open,
  -- TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(email.last_unsub_date)), DAY) / 30 AS months_since_last_ubsubscribe,
  -- avg(Outdoor_Living_pct), avg(Cargo_pct), avg(Soft_Cooler_pct),avg(Bags_pct),avg(Drinkware_pct),avg(Hard_Cooler_pct),avg(Other_pct)
  CASE 
    WHEN avg(Outdoor_Living_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Cargo_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Soft_Cooler_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Bags_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Drinkware_pct) <> 0 THEN 1 ELSE 0
  END + 
  CASE 
    WHEN avg(Hard_Cooler_pct) <> 0 THEN 1 ELSE 0
  END + 
  CASE 
    WHEN avg(Other_pct) <> 0 THEN 1 ELSE 0
  END as num_distinct_prod_cats
FROM yeti-dev-edw.aa_anon.aa_customer customer
#left join count_discount cd2 on cd2.yeti_customer_uuid= customer.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_order orders on orders.yeti_customer_uuid= customer.yeti_customer_uuid
#left join dis_prod dp on dp.yeti_customer_uuid= customer.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_order_item orderItem using(order_id)
left join Product_percentage pp on customer.yeti_customer_uuid=pp.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_email email  on customer.yeti_customer_uuid=email.yeti_customer_uuid
left join (
  SELECT yeti_customer_uuid, 
    count(order_id) as no_of_orders,  
    (AVG(TIMESTAMP_DIFF(TIMESTAMP(creation_date),TIMESTAMP(lag_order_date),DAY)))/30 as avg_time_btw_orders 
  FROM ( 
    SELECT yeti_customer_uuid, order_id,creation_date, 
      lag(creation_date) OVER (PARTITION BY yeti_customer_uuid ORDER BY creation_date) as lag_order_date  
    FROM `yeti-dev-edw.aa_anon.aa_order`  
  )
  GROUP BY yeti_customer_uuid
  HAVING count(order_id) >= 2
) subquery ON subquery.yeti_customer_uuid = customer.yeti_customer_uuid

where 
  orders.source_country = 'US' 
  AND orders.site_id = 'Yeti_US'

group by 
  customer.yeti_customer_uuid, 
  subquery.avg_time_btw_orders

having 
  count  (distinct orders.order_id) >=2

"""

query4 = """
select yeti_customer_uuid as customer_id,sum(case when discount<0 then quantity end) as discount_quantity
from yeti-dev-edw.aa_anon.aa_order_item oi
WHERE source_country = 'US' 
  AND site_id = 'Yeti_US'
group by yeti_customer_uuid
having 
#yeti_customer_uuid= "ce25cbdd-bcf9-4e0f-8ff7-9b1455c3804b"
#and
count  (distinct order_id) >=2
"""

query5 = """
SELECT *
FROM
  (
  SELECT 
    fo_prod_table.yeti_customer_uuid,
    fo_prod_table.first_order_id, 
    fo_prod_table.price,
    fo_prod_table.product_category,
    fo_prod_table.product_spend,
    DENSE_RANK() OVER(PARTITION BY fo_prod_table.yeti_customer_uuid ORDER BY fo_prod_table.product_spend DESC) as rnk1
  FROM
    (
    SELECT 
      fo_id.yeti_customer_uuid,
      fo_id.first_order_id, 
      oi_prod_table.product_id, 
      oi_prod_table.product_category, 
      oi_prod_table.price,
      oi_prod_table.product_spend,
    FROM
      (SELECT
        orders.yeti_customer_uuid,
        COUNT(orders.order_id) as order_cnt,
        MIN(orders.order_id) as first_order_id,
        MIN(orders.creation_date) first_order_date,
      FROM `yeti-dev-edw.aa_anon.aa_order` orders
      WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
      GROUP BY orders.yeti_customer_uuid
      HAVING order_cnt > 1) fo_id
    LEFT JOIN
      (
      SELECT
        order_item.yeti_customer_uuid,
        order_item.order_id,
        order_item.order_item_id,
        order_item.product_id, 
        product.product_category, 
        product.price, 
        order_item.quantity*product.price as product_spend,
      FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
      LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
      ON order_item.product_id = product.product_id) oi_prod_table
    ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
    ) fo_prod_table
  )
WHERE rnk1 = 1
ORDER BY yeti_customer_uuid
"""



job1 = client.query(query1)
job2 = client.query(query2)
job3 = client.query(query3)
jobM = client.query(Main_query)
job4 = client.query(query4)
job5 = client.query(query5)

df_has_discount = job1.to_dataframe()
df_color_count = job2.to_dataframe()
df_lifetime_quantity = job3.to_dataframe()
df= jobM.to_dataframe()
df_discount_quantity = job4.to_dataframe()
df_entry_product_category = job5.to_dataframe()

In [ ]:
df_merged_2 = pd.merge(df, df_has_discount, on= 'customer_id',how = 'left')
df_merged_2 = pd.merge(df_merged_2, df_color_count, on= 'customer_id', how = 'left')
df_merged_2 = pd.merge(df_merged_2, df_lifetime_quantity, on='customer_id', how = 'left')

In [ ]:
df_merged_2

,customer_id,recency,avg_time_btw_orders,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,...,white,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,lifetime_quantity,per_order_quantity
0,87b97771-314c-4928-a113-506b01bc8687,16.733333,5.120833,57.833333,1074448.0,89.492587,0.000000,0.086957,0.000000,0.043478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,2.555556
1,e49ca1b6-882e-4d61-bc76-9808fbeb9a77,22.300000,1.483333,25.300000,893518.0,176.794222,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,6.333333
2,81b968f3-cefb-4d9a-9cad-7a8ea1d4d7b1,6.200000,5.825000,29.566667,306402.0,59.265377,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14842017-f2b6-4b63-9cb6-300678d133d0,10.200000,3.416667,17.066667,2617.0,523.400000,0.000000,0.000000,0.009346,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,35.666667
4,f0aae549-8a1f-4074-9092-aa3599294b1c,7.300000,12.344444,44.366667,379650.0,128.303481,0.000000,0.000000,0.230769,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,3.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,48767193-aa4f-4194-b4ea-fbe70a5ef8eb,8.400000,18.116667,44.633333,70901.0,55.175875,0.000000,0.000000,0.000000,0.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999996,8a563328-6ef8-4a76-973e-a2f14c0c61c5,2.933333,9.200000,30.600000,241673.0,132.787363,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,50720492-3348-4610-becd-1f8f10663a67,23.800000,15.783333,87.033333,160909.0,44.984344,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.400000
999998,4d7f046d-93ae-4459-a107-bd873b668f4a,10.333333,16.583333,43.533333,365.0,52.142857,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query1 = """
with discount as(
select order_id, 
  case
  when sum(discount)!=0 then 1
  else 0
  end as has_discount
from yeti-dev-edw.aa_anon.aa_order_item
group by order_id )

select o.yeti_customer_uuid as customer_id,sum(has_discount) as has_discount
from discount d
left join `yeti-dev-edw.aa_anon.aa_order` o ON d.order_id=o.order_id
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2
limit 1000000
"""

job = client.query(query1)
df_has_discount = job.to_dataframe()



In [ ]:
df_has_discount.head()

In [ ]:
query2 = """
with join_color as(
with color as (
SELECT  product_id,
(case when color="Black" then 1 else 0 end) as black,
(case when color="Navy"  then 1 else 0 end) as navy,
(case when color="White" then 1 else 0 end) as white,
(case when color="Seafoam" then 1 else 0 end) as seafoam,
(case when color="Stainless" then 1 else 0 end) as stainless,
(case when color="Charcoal" then 1 else 0 end) as charcoal,
(case when color="Nordic Purple" then 1 else 0 end) as nordic_purple,
(case when color="Harvest Red" then 1 else 0 end) as harvest_red,
(case when color="Alpine Yellow" then 1 else 0 end) as alpine_yellow,
(case when color not in("White","Black","Navy","Seafoam","Stainless" ,"Charcoal","Nordic Purple","Harvest Red","Alpine Yellow") then 1 else 0 end) as other_color
from  `yeti-dev-edw.aa_anon.aa_product` p
)


select oi.order_id,oi.order_item_id,
sum(black)*sum(quantity) as black
,sum(navy)*sum(quantity) as navy
,sum(white)*sum(quantity) as white
,sum(seafoam)*sum(quantity) as seafoam
,sum(stainless)*sum(quantity) as stainless
,sum(charcoal)*sum(quantity) as charcoal
,sum(nordic_purple)*sum(quantity) as nordic_purple
,sum(harvest_red)*sum(quantity) as harvest_red
,sum(alpine_yellow)*sum(quantity) as alpine_yellow
,sum(other_color)*sum(quantity) as other_color



from color c
left join  `yeti-dev-edw.aa_anon.aa_order_item` oi on c.product_id=oi.product_id
group by oi.order_id,oi.order_item_id)

select o.yeti_customer_uuid as customer_id, sum (black) as black
,sum(navy) as navy
,sum(white) as white 
,sum(seafoam) as seafoam
,sum(stainless) as stainless
,sum(charcoal) as charcoal
,sum(nordic_purple) as nordic_purple
,sum(harvest_red) as harvest_red
,sum(alpine_yellow) as alpine_yellow
,sum(other_color) as other_color

from join_color jc 
left join  `yeti-dev-edw.aa_anon.aa_order` o on o.order_id=jc.order_id
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2
limit 1000000
"""

job = client.query(query2)
df_color_count = job.to_dataframe()

In [ ]:
df_color_count.head()

In [ ]:

query3 = """
SELECT o.yeti_customer_uuid as customer_id, SUM(oi.quantity) as lifetime_quantity,    sum(oi.quantity) / count(distinct o.order_id) as per_order_quantity,
FROM `yeti-dev-edw.aa_anon.aa_order_item` oi
LEFT JOIN `yeti-dev-edw.aa_anon.aa_order` o ON oi.order_id=o.order_id
#LEFT JOIN yeti-dev-edw.aa_anon.aa_customer customer ON o.yeti_customer_uuid=customer.yeti_customer_uuid
WHERE o.source_country = 'US' 
  AND o.site_id = 'Yeti_US'
group by o.yeti_customer_uuid
having
  count  (distinct o.order_id) >=2
limit 1000000
"""

job = client.query(query3)
df_lifetime_quantity = job.to_dataframe()

In [ ]:
Main_query = """
with 

Product_percentage as (SELECT 
  customer.yeti_customer_uuid, 
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Other'or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Other' or  product.product_category = '(not set)' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Other_pct,
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Outdoor Living' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Outdoor_Living_pct,
  CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Drinkware' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Drinkware_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Cargo' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Cargo_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Soft Cooler' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Soft_Cooler_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Bags' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Bags_pct,

 CASE 
    WHEN SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) = 0 
    THEN 0 
    ELSE 
      CASE 
        WHEN SUM(orderItem.quantity) = 0 
        THEN 0 
        ELSE 
          CASE 
            WHEN SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) < 0 
            THEN -1 * ABS(SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END)) / SUM(orderItem.quantity) 
            ELSE SUM(CASE WHEN product.product_category = 'Hard Cooler' THEN orderItem.quantity ELSE 0 END) / SUM(orderItem.quantity) 
          END 
      END 
  END AS Hard_Cooler_pct,

FROM yeti-dev-edw.aa_anon.aa_customer customer
LEFT JOIN yeti-dev-edw.aa_anon.aa_order orders using(yeti_customer_uuid)
LEFT JOIN yeti-dev-edw.aa_anon.aa_order_item orderItem USING (order_id)
LEFT JOIN yeti-dev-edw.aa_anon.aa_product product USING (product_id)
WHERE product_category IS NOT NULL
GROUP BY yeti_customer_uuid)

SELECT 
  customer.yeti_customer_uuid as customer_id, 
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(orders.creation_date)), DAY) / 30 AS recency,
  subquery.avg_time_btw_orders as avg_time_btw_orders,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(min(orders.creation_date)), DAY) / 30 AS months_elapsed,
  #sum(quantity) as lifetime_quantity,
  #sum (case when orderItem.discount>0 then quantity else 0 end) as test,
  #sum(quantity) / count(orders.order_id) as per_order_quantity,
  round(sum(orders.product_total)) as lifetime_spent,
  round(sum(orders.product_total))/ count(orders.order_id) as per_order_spent,

  avg(Outdoor_Living_pct) as outdoor_equipment_pct,
  avg(Cargo_pct) as Cargo_pct,
  avg(Soft_Cooler_pct) as Soft_Cooler_pct,
  avg(Bags_pct) as Bags_pct,
  avg(Drinkware_pct) as drinkware_pct,
  avg(Hard_Cooler_pct) as Hard_Cooler_pct,
  avg(Other_pct) as other_pct,
  #avg(has_discount) as order_with_discount,

  #avg(case when (bd) is null then 0 else bd  end ) as bag_discount_used,
  #avg(case when (od) is null then 0 else od  end ) as others_discount_used,
  #avg(case when (old) is null then 0 else old  end ) as outdoor_living_discount_used,
  #avg(case when (dd) is null then 0 else dd  end ) as drinkware_discount_used,
  #avg(case when (cd) is null then 0 else cd  end ) as cargo_discount_used,
  #avg(case when (scd) is null then 0 else scd  end ) as soft_cooler_discount_used,
  #avg(case when (hcd) is null then 0 else hcd  end ) as hard_cooler_discount_used,

  count(distinct orders.order_id) as purchase_times,

  COUNT(email_id) as email_received,
  COUNT(CASE WHEN last_open_date IS NOT NULL THEN 1 END) / NULLIF(COUNT(email_id),0) AS email_open_rate,
  CASE WHEN COUNT(email_id) = 0 THEN NULL ELSE MAX(CASE WHEN unsub_count > 0 THEN 1 ELSE 0 END) END AS unsubscribed,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(email.last_open_date)), DAY) / 30 AS months_since_last_email_open,
  -- TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), TIMESTAMP(max(email.last_unsub_date)), DAY) / 30 AS months_since_last_ubsubscribe,
  -- avg(Outdoor_Living_pct), avg(Cargo_pct), avg(Soft_Cooler_pct),avg(Bags_pct),avg(Drinkware_pct),avg(Hard_Cooler_pct),avg(Other_pct)
  CASE 
    WHEN avg(Outdoor_Living_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Cargo_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Soft_Cooler_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Bags_pct) <> 0 THEN 1 ELSE 0 
  END + 
  CASE 
    WHEN avg(Drinkware_pct) <> 0 THEN 1 ELSE 0
  END + 
  CASE 
    WHEN avg(Hard_Cooler_pct) <> 0 THEN 1 ELSE 0
  END + 
  CASE 
    WHEN avg(Other_pct) <> 0 THEN 1 ELSE 0
  END as num_distinct_prod_cats
FROM yeti-dev-edw.aa_anon.aa_customer customer
#left join count_discount cd2 on cd2.yeti_customer_uuid= customer.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_order orders on orders.yeti_customer_uuid= customer.yeti_customer_uuid
#left join dis_prod dp on dp.yeti_customer_uuid= customer.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_order_item orderItem using(order_id)
left join Product_percentage pp on customer.yeti_customer_uuid=pp.yeti_customer_uuid
left join  yeti-dev-edw.aa_anon.aa_email email  on customer.yeti_customer_uuid=email.yeti_customer_uuid
left join (
  SELECT yeti_customer_uuid, 
    count(order_id) as no_of_orders,  
    (AVG(TIMESTAMP_DIFF(TIMESTAMP(creation_date),TIMESTAMP(lag_order_date),DAY)))/30 as avg_time_btw_orders 
  FROM ( 
    SELECT yeti_customer_uuid, order_id,creation_date, 
      lag(creation_date) OVER (PARTITION BY yeti_customer_uuid ORDER BY creation_date) as lag_order_date  
    FROM `yeti-dev-edw.aa_anon.aa_order`  
  )
  GROUP BY yeti_customer_uuid
  HAVING count(order_id) >= 2
) subquery ON subquery.yeti_customer_uuid = customer.yeti_customer_uuid

where 
  orders.source_country = 'US' 
  AND orders.site_id = 'Yeti_US'

group by 
  customer.yeti_customer_uuid, 
  subquery.avg_time_btw_orders

having 
  count  (distinct orders.order_id) >=2
limit 1000000
"""
job = client.query(Main_query)
df= job.to_dataframe()

In [ ]:
df.head()

In [8]:
data = pd.read_csv('data_all_merged_03_08_23_vfinal.csv')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2131520 entries, 0 to 2131519
Data columns (total 33 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   customer_id                   object 
 2   months_elapsed                float64
 3   lifetime_spent                float64
 4   per_order_spent               float64
 5   outdoor_equipment_pct         float64
 6   Cargo_pct                     float64
 7   Soft_Cooler_pct               float64
 8   Bags_pct                      float64
 9   drinkware_pct                 float64
 10  Hard_Cooler_pct               float64
 11  other_pct                     float64
 12  purchase_times                int64  
 13  recency                       float64
 14  avg_time_btw_orders           float64
 15  email_received                int64  
 16  email_open_rate               float64
 17  unsubscribed                  float64
 18  months_since_last_emai

In [28]:
data.isna().sum()

Unnamed: 0                           0
customer_id                          0
months_elapsed                       0
lifetime_spent                       0
per_order_spent                      0
outdoor_equipment_pct             2594
Cargo_pct                         2594
Soft_Cooler_pct                   2594
Bags_pct                          2594
drinkware_pct                     2594
Hard_Cooler_pct                   2594
other_pct                         2594
purchase_times                       0
recency                              0
avg_time_btw_orders                  0
email_received                       0
email_open_rate                 383955
unsubscribed                    383955
months_since_last_email_open    546434
num_distinct_prod_cats               0
lifetime_quantity                    5
per_order_quantity                   5
has_discount                         5
black                            11850
navy                             11850
white                    

In [29]:
data.columns

Index(['Unnamed: 0', 'customer_id', 'months_elapsed', 'lifetime_spent',
       'per_order_spent', 'outdoor_equipment_pct', 'Cargo_pct',
       'Soft_Cooler_pct', 'Bags_pct', 'drinkware_pct', 'Hard_Cooler_pct',
       'other_pct', 'purchase_times', 'recency', 'avg_time_btw_orders',
       'email_received', 'email_open_rate', 'unsubscribed',
       'months_since_last_email_open', 'num_distinct_prod_cats',
       'lifetime_quantity', 'per_order_quantity', 'has_discount', 'black',
       'navy', 'white', 'seafoam', 'stainless', 'charcoal', 'nordic_purple',
       'harvest_red', 'alpine_yellow', 'other_color'],
      dtype='object')